In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
import yfinance as yf
import time

In [10]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

In [11]:
model = SGDRegressor(learning_rate='constant', eta0=0.01)
scaler = StandardScaler()

In [12]:
def fetch_stock_data(ticker: str, interval: str, iterations: int):
    i=0

    while True:
        # Fetch latest stock data
        stock_data = yf.download(ticker, period="1d", interval=interval)

        # Process the last data point and create a copy
        last_last_last_data = stock_data.tail(3).head(1).copy()
        last_last_data = stock_data.tail(2).head(1).copy()
        last_data = stock_data.tail(1).copy()
        #print(last_data)

        if not last_last_last_data.empty or not last_last_data.empty:
            # Get last last values
            close_last_last_price = last_last_data["Close"].iloc[-1]
            volume_last_last_value = last_last_data["Volume"].iloc[-1]

            #Get last last last values
            close_last_last_last_price = last_last_last_data["Close"].iloc[-1]
            volume_last_last_last_value = last_last_last_data["Volume"].iloc[-1]

            price_last_last_change = (close_last_last_price - close_last_last_last_price)
            volume_last_last_change = (volume_last_last_value - volume_last_last_last_value) / volume_last_last_value

            # Standardize the features
            features_scaled = scaler.fit_transform(volume_last_last_change.reshape(-1,1))

            last_close_volume = last_data["Volume"].iloc[-1]
            volume_change_to_be_predicted = (last_close_volume - volume_last_last_value) / volume_last_last_value
            feature_to_be_predicted = scaler.fit_transform(volume_change_to_be_predicted.reshape(-1,1))
            # Train or predict with the model
            y = price_last_last_change
            model.partial_fit(features_scaled, [y])
            # Predict next value and print
            pred = model.predict(feature_to_be_predicted.reshape(1,-1))
            last_close = last_data["Close"].iloc[-1]
            print("")
            print(f"true: {last_close}")
            print(f"Predicted: {pred[0]+close_last_last_price}")

        i += 1
        if i>=iterations:
            break

        # Wait before fetching the next data point (depending on the interval)
        if interval == "1m":
            time.sleep(60)
        elif interval == "5m":
            time.sleep(300)
        # Add more cases as needed for other intervals


In [13]:
# Example usage
if __name__ == "__main__":
    ticker_symbol = "AMZN"  # Example ticker symbol for Apple Inc.
    time_interval = "1m"  # Fetch data at 1-minute intervals
    stream_iterations = 10# Stream data for 10 minutes

    fetch_stock_data(ticker_symbol, time_interval, stream_iterations)

[*********************100%%**********************]  1 of 1 completed



true: 186.78500366210938
Predicted: 186.7948487854004


[*********************100%%**********************]  1 of 1 completed



true: 186.80999755859375
Predicted: 186.769209715271


[*********************100%%**********************]  1 of 1 completed


...


true: 186.80999755859375
Predicted: 186.73390549273424


[*********************100%%**********************]  1 of 1 completed



true: 186.7899932861328
Predicted: 186.81977322491625


[*********************100%%**********************]  1 of 1 completed


true: 186.78399658203125
Predicted: 186.79684343700305
